In [1]:
from threading import Thread
import json
from stmpy import Machine, Driver
import paho.mqtt.client as mqtt
from utils import BROKER, PORT, TOPIC


class MQTT_Client_1:
    def __init__(self, stm):
        self.state_machine = stm
        self.count = 0
        self.client = mqtt.Client()
        self.client.on_connect = self.on_connect
        self.client.on_message = self.on_message
        self.stm_driver: Driver = None

    def on_connect(self, client, userdata, flags, rc):
        print("on_connect(): {}".format(mqtt.connack_string(rc)))

    def on_message(self, client, userdata, msg):
        try:
            message = json.loads(msg.payload.decode('utf-8'))["msg"]
        except json.decoder.JSONDecodeError:
            return
                    
        print("on_message(): topic: {}".format(msg.topic))

        if message == "Valid code":
            self.stm_driver.send("correct_code", "student")

        else: self.stm_driver.send("wrong_code", "student")
    
        
    def start(self, broker, port):
        print("Connecting to {}:{}".format(broker, port))
        self.client.connect(broker, port)
        self.client.subscribe(TOPIC)

        try:
            thread = Thread(target=self.client.loop_forever)
            thread.start()
        except KeyboardInterrupt:
            print("Interrupted")
            self.client.disconnect()

In [2]:
!pip3 install paho-mqtt
from stmpy import Driver, Machine
from threading import Thread
import paho.mqtt.client as mqtt
import json
from stmpy import Machine, Driver
import ipywidgets as widgets
from IPython.display import display
import random


#NOTES:
## Håndtere i server som mottar join-koder at den enten sender trigger: riktig kode, eller trigger: feil kode, 
# utifra det så vet vi hvilken state vi skal tils
#Hver eneste student må ha en slags kode/gruppenavn


class Student:
      def __init__(self):
            self.button_create = widgets.Button(description="Join Session")
            self.button_create.on_click(self.on_button_join)

            self.mqtt_client: mqtt.Client = None
            self.stm: Machine = None
            
            # text field
            self.stu_code = widgets.Text(value='', placeholder='', description='student code:', disabled=False)
            print(self.stu_code.value)
            display(self.button_create, self.stu_code)
            
            self.group_name = widgets.Text(value='', placeholder='', description='Group name:', disabled=False)
            display(self.button_create, self.group_name)

            display()

      def on_wait(self):
            """TODO"""


      
      def ask_help(self):
            question_for_server_use = self.name + " # " + self.question
            self.mqtt_client.publish(TOPIC, question_for_server_use)

      def finish_task(self):
           param = self.name + " $ " + self.question
           self.mqtt_client.publish('update', param)
           self.question += 1      

      

      def publish(self):
            print(self.stu_code)
            code = self.stu_code.value
            group_name = self.group_name.value
            
            self.mqtt_client.publish(TOPIC, code + " # " + group_name)
            self.stu_code.value = "Loading"
      
      def setup_lab(self):
            self.button_help = widgets.Button(description="Get help")
            self.button_help.on_click(self.ask_help)
            display(self.button_help)
            pass

      def save_text(self):
           self.code = self.stu_code
           

      def on_button_join(self, session_code_stu):
        self.stm.send('join') # <---- here we send a message
        print(self.stm.state)
        pass


# initial transition
t0 = {'source': 'initial',
      'target': 'idle'}

# transitions

#t1 and t2 are transitions from idle to wait and back
t1 = {'trigger':'join', 
      'source':'idle', 
      'target':'wait'}


t2 = {'trigger':'wrong_code', 
      'source':'wait', 
      'target':'idle'}


t3 = {'trigger':'correct_code', 
      'source':'wait', 
      'target':'lab_session_active'}



t4 = {'trigger':'ask_help', 
      'source':'lab_session_active', 
      'target':'lab_session_active',
      'effect': 'ask_help' }

t5 = {'trigger':'leave_queue', 
      'source':'lab_session_active', 
      'target':'lab_session_active'}

t6 = {'trigger':'finish_task', 
      'source':'lab_session_active', 
      'target':'lab_session_active'}

t7 = {'trigger':'queue_request', 
      'source':'lab_session_active', 
      'target':'lab_session_active'}


t8 = {'trigger':'leave_lab', 
      'source':'lab_session_active', 
      'target':'exit'}




# the states:
idle = {'name': 'idle'}

wait = {'name': 'wait',
        'entry': 'publish'}

lab_session_active = {'name': 'lab_session_active',
                      'entry': 'setup_lab'}

    

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [3]:
student = Student()
state_machine = Machine(transitions=[t0, t1, t2, t3, t4, t5, t6, t7, t8], states=[idle, wait, lab_session_active], obj=student, name="student")
student.stm = state_machine

driver = Driver()
driver.add_machine(state_machine)

myclient = MQTT_Client_1(student)
student.mqtt_client = myclient.client
myclient.stm_driver = driver

driver.start()
myclient.start(BROKER, PORT)

Button(description='Join Session', style=ButtonStyle())

Text(value='', description='student code:', placeholder='')

Button(description='Join Session', style=ButtonStyle())

Text(value='', description='Group name:', placeholder='')

Connecting to mqtt20.iik.ntnu.no:1883


on_connect(): Connection Accepted.
